In [ ]:
"""
Evaluation
"""

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import roc_auc_score
import torchvision.datasets as dset
from utils.deepsvdd import DeepSVDD_network, pretrain_autoencoder, TrainerDeepSVDD
import os


In [ ]:
# Specify the cuda setting if you needed
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# IMPORTANT
# PLEASE CHANGE THE VAULUE OF DeepTaster_forder TO THE DEEPTASTER DIRECTORY 

Home_directory = {}
DeepTaster_forder = Home_directory+"/DeepTaster/"

In [ ]:

dataset = dset.ImageFolder(root=DeepTaster_forder +'images/test',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

test_dataloader = torch.utils.data.DataLoader(dataset,
                                          batch_size=16,
                                          shuffle=False,
                                          num_workers=8)

In [ ]:
net = DeepSVDD_network().to(device)
state_dict = torch.load(DeepTaster_forder +'models/classifier/pretrained.pth')
c = torch.Tensor(state_dict['center']).to(device)
net=torch.load(DeepTaster_forder +'models/classifier/deepsvdd.pt')
threshold_file=open(DeepTaster_forder +'models/classifier/threshold.txt','r')
threshold=float(threshold_file.readline())
threshold_file.close()

In [ ]:
# NOTE THAT THE ORDER OF RESULTS IS THREE CIFAR10 MODELS > THREE MNIST MODELS > THREE TINY-IMAGENET MODELS
# GROUND TRUTH RESULTS: STOLEN, STOLEN, STOLEN, BENIGN, BENIGN, BENIGN, BENIGN, BENIGN, BENIGN
net.eval()
print('Testing...')
temp=0
test=0
attack_probability=0
with torch.no_grad():
    for x, y in test_dataloader:
        x = x.float().to(device)
        z = net(x)
        score = torch.sum((z - c) ** 2, dim=1)
        for i in range(16):
          test+=1
          temp+=1
     
          if score[i]<=threshold:
            attack_probability+=1
          if temp==288:
            if attack_probability>144:
              print("Stolen!! with Attack_pobability:", round(attack_probability/288,3))
            else:
              print("Benign!! with Attack_pobability:", round(attack_probability/288,3))
            temp=0
            attack_probability=0
      